---

## Mise en place d'une **validation croisée imbriquée** avec un jeu de test final

1. **Division initiale des données** : Diviser les données en deux parties :
   - **80% des données pour l'entraînement et la validation croisée imbriquée**.
   - **20% des données pour un test final** que le modèle ne verra qu'à la toute fin.

2. **Validation croisée imbriquée sur les 80% d’entraînement** : Utiliser une validation croisée interne pour optimiser les hyperparamètres du modèle. Cela permet de tester plusieurs combinaisons de paramètres tout en évitant d'exposer le modèle aux données de test final.

3. **Évaluation finale** : Une fois le modèle optimisé sur les 80% des données avec validation croisée, l’évaluer sur les 20% restants pour obtenir un score de performance réaliste et indépendant.

---

---

# Notebook : Optimisation Avancée avec Validation Croisée Imbriquée et Test Final pour Random Forest

**Objectifs :**
1. Diviser les données en ensembles d’entraînement et de test.
2. Effectuer une **validation croisée imbriquée** sur l’ensemble d’entraînement (80%) pour optimiser les hyperparamètres.
3. Évaluer la performance finale du modèle optimisé sur le **jeu de test final** (20%).

---

## Étape 1 : Importer les bibliothèques nécessaires et charger les données

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os
import json
import numpy as np

---

## Étape 2 : Diviser les données en ensembles d’entraînement et de test

Nous réservons 20% des données pour un test final, tandis que les 80% restants serviront pour l’entraînement et la validation croisée imbriquée.

In [2]:
# Charger les données complètes
file_path = 'note-books/phase_2_training/04_correlation_analysis/data_final_features.csv'
df = pd.read_csv(file_path)

# Encoder `nutriscore_grade` en valeurs numériques
df['nutriscore_grade_encoded'] = df['nutriscore_grade'].map({'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4})

# Préparer les variables explicatives et la cible
X = df[['fat_100g', 'saturated-fat_100g', 'energy_100g', 'energy-kcal_100g', 'sugars_100g']]
y = df['nutriscore_grade_encoded']

# Diviser les données en ensembles d'entraînement et de test (80% pour entraînement, 20% pour test final)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

---

## Étape 3 : Définir la pipeline et les hyperparamètres pour RandomizedSearchCV sur l’ensemble d’entraînement

Nous allons configurer une pipeline et une grille de recherche pour optimiser les hyperparamètres du modèle sur l’ensemble d’entraînement (80%).

In [3]:
# Créer une pipeline avec une étape de standardisation et le modèle Random Forest
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(random_state=42))
])

# Définir une grille de recherche avec des valeurs d'hyperparamètres restreintes
param_dist = {
    'model__n_estimators': [50, 100, 150],
    'model__max_depth': [10, 15, 20],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Configurer RandomizedSearchCV pour la validation croisée imbriquée
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=10,
    scoring='neg_mean_squared_error',
    cv=5,  # Validation croisée à 5 plis sur l’ensemble d’entraînement
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# Entraîner le modèle avec Randomized Search sur l'ensemble d'entraînement (80%)
random_search.fit(X_train, y_train)

# Extraire les meilleurs paramètres
best_params = random_search.best_params_
print("Meilleurs paramètres trouvés avec Random Search :", best_params)

# Sauvegarder les meilleurs paramètres pour un suivi ultérieur
output_folder = 'note-books/phase_2_training/10_random_forest_with_final_test'
os.makedirs(output_folder, exist_ok=True)
best_params_path = f'{output_folder}/best_hyperparameters.json'
with open(best_params_path, 'w') as f:
    json.dump(best_params, f)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END model__max_depth=15, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  20.8s
[CV] END model__max_depth=15, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  20.7s
[CV] END model__max_depth=15, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  20.9s
[CV] END model__max_depth=15, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  20.9s
[CV] END model__max_depth=15, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=  20.9s
[CV] END model__max_depth=20, model__min_samples_leaf=2, model__min_samples_split=5, model__n_estimators=150; total time=  36.5s
[CV] END model__max_depth=20, model__min_samples_leaf=2, model__min_samples_split=5, model__n_estimators=150; total time=  36.6s
[CV] END model__max_depth=20, model_

---

## Étape 4 : Évaluation finale sur le jeu de test

Nous utilisons maintenant l'ensemble de test (20%) pour une évaluation finale du modèle optimisé, ce qui donnera une estimation réaliste de ses performances.


In [4]:
# Utiliser le meilleur modèle trouvé par Random Search
best_model = random_search.best_estimator_

# Prédire sur le jeu de test final
y_pred = best_model.predict(X_test)

# Calculer les métriques de performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Évaluation finale sur le jeu de test :")
print(f"Mean Squared Error (MSE) : {mse}")
print(f"Coefficient de détermination (R²) : {r2}")

# Vérification de la performance du modèle
target_r2 = 0.85  # Objectif réaliste pour le R²

if r2 >= target_r2:
    print(f"Le modèle atteint le seuil de performance souhaité sur le jeu de test final (R² ≥ {target_r2}).")
else:
    print("Le modèle n'atteint toujours pas le seuil de performance souhaité.")

Évaluation finale sur le jeu de test :
Mean Squared Error (MSE) : 0.5514624002011859
Coefficient de détermination (R²) : 0.6948232057588746
Le modèle n'atteint toujours pas le seuil de performance souhaité.


---

## Étape 5 : Sauvegarder le modèle optimisé et ses paramètres

Si les performances sont satisfaisantes, nous sauvegardons le modèle optimisé pour les étapes finales.

In [5]:
# Chemin de sauvegarde pour le modèle optimisé
model_path_optimized = f'{output_folder}/Random_Forest_final_optimized.joblib'

# Sauvegarder le modèle optimisé
joblib.dump(best_model, model_path_optimized)
print(f"Modèle Random Forest optimisé sauvegardé sous : {model_path_optimized}")

Modèle Random Forest optimisé sauvegardé sous : note-books/phase_2_training/10_random_forest_with_final_test/Random_Forest_final_optimized.joblib



---

### Explication des étapes

1. **Division des données** : En gardant 20% pour un test final, on réserve une portion de données non vue pour évaluer la généralisation réelle du modèle.
2. **Validation croisée imbriquée** : La validation croisée sur l’ensemble d’entraînement (80%) optimise les hyperparamètres sans utiliser le jeu de test final.
3. **Évaluation finale sur le jeu de test** : Cette étape fournit une estimation de la performance du modèle sur des données indépendantes.

---

### Résumé

Dans ce notebook, nous avons :
1. Divisé les données en **ensembles d’entraînement (80%) et de test (20%)** pour assurer une évaluation finale indépendante.
2. Utilisé **RandomizedSearchCV** pour optimiser les hyperparamètres avec une validation croisée imbriquée sur l’ensemble d’entraînement.
3. Évalué le modèle optimisé sur le jeu de test final pour obtenir une estimation réaliste de ses performances.
4. Sauvegardé le modèle si les performances répondent aux critères souhaités (par exemple, R² ≥ 0.85).

Cela permet une optimisation des hyperparamètres avec une estimation finale fiable sans compromettre la séparation entre entraînement et test.